In [ ]:
%reload_ext autoreload
%autoreload 2

import cv2

import os
import sys
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *

from collections import defaultdict

from matplotlib.path import Path
%matplotlib inline

import pandas as pd

from joblib import Parallel, delayed

from scipy.interpolate import RectBivariateSpline
from skimage.transform import rescale, resize

In [ ]:
stack = 'MD589'
# stack = 'MD594'
dm = DataManager(stack=stack)

label_polygons = pd.read_hdf('/home/yuncong/Brain/database/'+stack+'_label_polygons.h5', 'label_polygons')

In [ ]:
labels =  ['BackG', '5N', '7n', '7N', '12N', 'Gr', 'LVe', 'Pn', 'SuVe', 'VLL']

other_labels = [l for l in label_polygons.dropna().keys() if l not in labels]

# label_dict = dict([(l,i) for i, l in enumerate(labels)])

label_dict = dict([(l,i) for i, l in enumerate(labels)] + \
                  zip(other_labels, range(len(labels), len(labels)+len(other_labels))))

colors = (np.loadtxt(os.environ['REPO_DIR'] + '/visualization/100colors.txt')*255).astype(np.uint8)
colors[label_dict['BackG']] = 255

In [ ]:
labelMapLossless_dir = '/oasis/projects/nsf/csd395/yuncong/CSHL_labelMapsLossless/%(stack)s/' % {'stack': stack}
if not os.path.exists(labelMapLossless_dir):
    os.makedirs(labelMapLossless_dir)

In [ ]:
label_polygons

In [ ]:
from matplotlib.path import Path
from itertools import product

In [ ]:
# only 9 classes
for sec in label_polygons.index:
    print sec
    
    available_labels = label_polygons.loc[sec].dropna().index
    
    valid_labels = set(available_labels) & set(labels)
    
    if len(valid_labels) == 0:
        continue
    
    label_map = np.zeros((dm.image_height, dm.image_width), np.int8)
    for l in valid_labels:
        print l
        vs = label_polygons[l][sec]
        xmin, ymin = np.min(vs, axis=0)
        xmax, ymax = np.max(vs, axis=0)
                
        if xmin > dm.image_width - 1 or ymin > dm.image_height - 1:
            print 'min coordinates exceed image boundary'
            continue
        
        xmax = min(xmax, dm.image_width-1)
        ymax = min(ymax, dm.image_height-1)
        
#         import time
#         t = time.time()
        
#         points_to_test = np.array(list(product(range(ymin, ymax+1), range(xmin, xmax+1))))[:,::-1]
        xs, ys = np.meshgrid(range(xmin, xmax+1), range(ymin, ymax+1))
        points_to_test = np.c_[xs.flat, ys.flat]
        
#         print time.time() - t
        
        is_contained = Path(vs).contains_points(points_to_test)
        label_map[points_to_test[is_contained, 1], points_to_test[is_contained, 0]] = label_dict[l]
    
    
    bp.pack_ndarray_file(label_map, os.path.join(labelMapLossless_dir, 
                                     '%(stack)s_%(sec)04d_labelMapLossless.bp' % \
                                    {'stack': stack, 'sec': sec}))

#     np.savez_compressed(os.path.join(labelMapLossless_dir, 
#                                      '%(stack)s_%(sec)04d_labelMapLossless' % \
#                                     {'stack': stack, 'sec': sec}), 
#                         label_map)

In [ ]:
# all landmarks
for sec in label_polygons.index:
    
    print sec
    
    available_labels = label_polygons.loc[sec].dropna().index
    
    valid_labels = set(available_labels)
    
    if len(valid_labels) == 0:
        continue
    
    label_map = np.zeros((dm.image_height, dm.image_width), np.int8)
    for l in valid_labels:
        print l
        vs = label_polygons[l][sec]
        xmin, ymin = np.min(vs, axis=0)
        xmax, ymax = np.max(vs, axis=0)
        
        if xmin > dm.image_width - 1 or ymin > dm.image_height - 1:
            print 'min coordinates exceed image boundary'
            continue
        
        xmax = min(xmax, dm.image_width-1)
        ymax = min(ymax, dm.image_height-1)
        
        points_to_test = np.array(list(product(range(ymin, ymax+1), range(xmin, xmax+1))))[:,::-1]
        is_contained = Path(vs).contains_points(points_to_test)
        
        label_map[points_to_test[is_contained, 1], 
                  points_to_test[is_contained, 0]] = label_dict[l]
    
    np.savez_compressed(os.path.join(labelMapLossless_dir, 
                                     '%(stack)s_%(sec)04d_labelMapLossless_AllClasses' % \
                                    {'stack': stack, 'sec': sec}), 
                        label_map)